In [14]:
import os 
import pandas as pd
from collections import defaultdict
import numpy as np 

Step 1. Get the all sample experiement longlist: 
Use cut and unique in bash - (could be done in the notebook) 



Step 2. Get the metadata and peptides: 
Done in bash with the command below 

Step 3. Get the formating done within this notebook 


# Preprocess the annotation 

In [15]:
ann_path = '/cluster/work/grlab/projects/projects2020_OHSU/annotation/gencode.v32.annotation.gtf'

In [16]:
# From Immunopepper preprocess.py 
def attribute_item_to_dict(a_item, file_type, feature_type):
    """  From attribute item in annotation file to get corresponding dictionary

    Parameters
    ----------
    a_item: str. attribute item
    file_type: str. Choose from {'gtf', 'gff', 'gff3'}
    feature_type: str. Extract other fields. We only
        consider 'CDS', 'mRNA' and 'transcript'

    Returns
    -------
    gtf_dict: dict. store all the necessary data

    """    
    
    gtf_dict = {}
    if file_type.lower() == 'gtf':
        attribute_list = a_item.split('; ')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split(' ')
            gtf_dict[pair[0]] = pair[1][1:-1]
    elif file_type.lower() == 'gff3':
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gtf_dict[pair[0]] = pair[1]
    elif file_type.lower() == 'gff':
        gff_dict = {}
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gff_dict[pair[0]] = pair[1]  # delete "", currently now work on level 2
        if feature_type == 'CDS':
            gtf_dict['transcript_id'] = gff_dict['Parent']
        elif feature_type in {'mRNA', 'transcript'}:  # mRNA or transcript
            gtf_dict['gene_id'] = gff_dict['geneID']
            gtf_dict['transcript_id'] = gff_dict['ID']
            gtf_dict['gene_type'] = gff_dict['gene_type']
            gtf_dict['transcript_type'] = gff_dict['transcript_type']

    return gtf_dict

In [17]:
#  From Immunopepper preprocess.py 
def leq_strand(coord1, coord2, strand):
    if strand == "+":
        return coord1 <= coord2
    else:
        return coord1 >= coord2


In [18]:
# From Immunopepper preprocess.py 
transcript_to_gene_dict = {}    # transcript -> gene id


gene_to_transcript_dict = {}    # gene_id -> list of transcripts
gene_cds_begin_dict = {}        # gene -> list of first CDS exons

transcript_to_cds_dict = {}     # transcript -> list of CDS exons
transcript_cds_begin_dict = {}  # transcript -> first exon of the CDS
transcript_to_strand = {}

file_type = ann_path.split('.')[-1]
chromesome_set = set()
# collect information from annotation file
for line in open(ann_path, 'r'):
    if line[0] == '#':
        continue
    item = line.strip().split('\t')
    chromesome_set.add(item[0])
    feature_type = item[2]
    attribute_item = item[-1]
    attribute_dict = attribute_item_to_dict(attribute_item, file_type, feature_type)
    # store relationship between gene ID and its transcript IDs
    if feature_type in ['transcript', 'mRNA']:
        gene_id = attribute_dict['gene_id']
        transcript_id = attribute_dict['transcript_id']
        if attribute_dict['gene_type'] != 'protein_coding' or attribute_dict['transcript_type']  != 'protein_coding':
            continue
        assert (transcript_id not in transcript_to_gene_dict)
        transcript_to_gene_dict[transcript_id] = gene_id
        if gene_id in gene_to_transcript_dict and transcript_id not in gene_to_transcript_dict[gene_id]:
            gene_to_transcript_dict[gene_id].append(transcript_id)
        else:
            gene_to_transcript_dict[gene_id] = [transcript_id]
        # Todo python is 0-based while gene annotation file(.gtf, .vcf, .maf) is one based
    elif feature_type == "CDS":
        parent_ts = attribute_dict['transcript_id']
        strand_mode = item[6]
        cds_left = int(item[3])-1
        cds_right = int(item[4])
        frameshift = int(item[7])
        transcript_to_strand[parent_ts] = strand_mode
        if parent_ts in transcript_to_cds_dict:
            transcript_to_cds_dict[parent_ts].append((cds_left, cds_right, frameshift))
        else:
            transcript_to_cds_dict[parent_ts] = [(cds_left, cds_right, frameshift)]
        if strand_mode == "+" :
            cds_start, cds_stop = cds_left, cds_right
        else:
            cds_start, cds_stop = cds_right, cds_left

        # we only consider the start of the whole CoDing Segment
        if parent_ts not in transcript_cds_begin_dict or \
           leq_strand(cds_start, transcript_cds_begin_dict[parent_ts][0], strand_mode):
            transcript_cds_begin_dict[parent_ts] = (cds_start, cds_stop, item)

# collect first CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    if target_gene not in gene_cds_begin_dict:
        gene_cds_begin_dict[target_gene] = []
    if ts_key in transcript_cds_begin_dict:
        gene_cds_begin_dict[target_gene].append(transcript_cds_begin_dict[ts_key])


In [19]:
# NEED TO SORT THE TRANSCRIPT TO CDS 

In [20]:
# Custom collection of CDS 
transcript_cds_begin_dict_bis = {}
transcript_cds_end_dict_bis = {}

gene_cds_begin_dict_bis = defaultdict(list)
gene_cds_end_dict_bis = defaultdict(list)

# will be in reading order 
for ts_key in transcript_to_cds_dict:
    if transcript_to_strand[ts_key] == '+': # '+'
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][0],
                                                 transcript_to_cds_dict[ts_key][0][1], '+')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][0],
                                                 transcript_to_cds_dict[ts_key][-1][1], '+')
        

    else: 
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][1],
                                                 transcript_to_cds_dict[ts_key][0][0], '-')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][1],
                                                 transcript_to_cds_dict[ts_key][-1][0], '-')
    
    assert(transcript_cds_begin_dict_bis[ts_key][0] == transcript_cds_begin_dict[ts_key][0])
    assert(transcript_cds_begin_dict_bis[ts_key][1] == transcript_cds_begin_dict[ts_key][1])

# collect first, last CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    gene_cds_begin_dict_bis[target_gene].append(transcript_cds_begin_dict_bis[ts_key])
    gene_cds_end_dict_bis[target_gene].append(transcript_cds_end_dict_bis[ts_key])

In [21]:
#gene_cds_begin_dict_bis

# Format the peptide raw file (from step 2)

In [22]:
# fields_meta_peptide_dict = ['peptide', 'id', 'readFrame', 'geneName', 'geneChr', 'geneStrand',
#                                 'mutationMode',
#                                 'junctionAnnotated', 'hasStopCodon', 'isInJunctionList',
#                                 'isIsolated', 'variantComb', 'variantSegExpr', 'modifiedExonsCoord',
#                                 'originalExonsCoord', 'vertexIdx',
#                                 'kmerType']

In [23]:
file_meta = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/TCGA-A2-A0D2-01A-21R-A034-07.all_d4aee54_GTEXcore_kmer_peptides_raw.tsv'

In [35]:
file_save = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/G_TCGA-A2-A0D2-01A-21R-A034-07_query_peptides.fa'


In [38]:
## Format and create flags 
pep_idx = 0 

with open(file_save, 'a') as sp:
    with open(file_meta, 'r') as fp:
        print("open {}".format(file_meta))
        lines = fp.readlines()
        ### Iterate over kmers 
        for line in lines:
            if 'modifiedExonsCoord' in line: #header
                continue
            line = line.replace('3-exons_9-mer ', '3-exons_9-mer@').replace('2-exons ', '2-exons@')
            kmer = line.split(',')[0]
            peptides = ','.join(line.split(',')[1:])
            peptides = peptides.split('@')
            ### Iterate over kmer-associated peptides
            for pep in peptides:
                pep = pep.split(',')
                pep_seq = pep[0]
                pep_gene = pep[3]
                pep_orig_coordS = pep[13].split('/')
                pep_modif_coordS = pep[14]
                strand = pep[5]
                has_stop_codon = pep[8]
                if has_stop_codon == '0':
                    break 
                ### Iterate over original genomic coordinates for include flag
                for pep_orig_coord in pep_orig_coordS:
                    pep_orig_coord = pep_orig_coord.split(';')
                    pep_orig_coord = [coord for coord in pep_orig_coord if coord != 'None']

                    start_cds = [ first_exon[0] for first_exon in 
                                 gene_cds_begin_dict_bis[pep_gene] ] 
                    end_cds = [ last_exon[1] for last_exon in 
                               gene_cds_end_dict_bis[pep_gene] ] 
                    ## Get peptide end and start coordinates
                    if strand == '+': # Do - strand 
                        pep_start = np.int(pep_orig_coord[0])
                        pep_end = np.int(pep_orig_coord[-1])
                    else: 
                        pep_start = np.int(pep_orig_coord[1])
                        pep_end = np.int(pep_orig_coord[-2])

                    ## Use end and start coordinates for 3' 5' include flag
                    if pep_start in start_cds: # We will always miss things that are new in the graph 
                        pep_5include = 1
                    else: 
                        pep_5include = 0 
                    if (pep_end in end_cds) or (has_stop_codon) == '1':
                        pep_3include = 1
                    else: 
                        pep_3include = 0 

                ### Iterate over original genomic coordinates for the junction position flag and between codon flag
                for pep_modi_coord in pep_modif_coordS.split('/'):
                    pep_idx +=1
                    pep_modi_coord = pep_modi_coord.split(';')
                    pep_modi_coord = [coord for coord in pep_modi_coord if coord != 'None']
                    tot_len = 0 
                    jx_list = []
                    shift = 0 
                    shift_list = []
                    ## Get nt length of each exon involved -> jx_list, shift_list
                    for pair in np.arange(0, len(pep_modi_coord), 2):
                        cds = int(pep_modi_coord[pair + 1]) - int(pep_modi_coord[pair])  # 0 based, open right 
                        cds += shift 
                        shift = cds % 3
                        jx_list.append(cds - shift)
                        shift_list.append(shift)
                    kmer_len = 9 

    #                 for start_pos in np.arange(0, len(pep_seq) - (kmer_len - 1) ):
    #                     if pep_seq[start_pos : start_pos + kmer_len ] == kmer:
    #                         pos_kmer = start_pos # 0 based, real position = pos_kmer +1
    #                 assert(pos_kmer == jx_list[0]/ 3)

                    ## Get aa position of the junction
                    if shift_list[0]: # junction is inside an amino acid
                        aa_junction_pos = int((jx_list[0] / 3)) # because 0 based
                        between_codons = 0 
                    else: # junction is between amino acids 
                        aa_junction_pos = int((jx_list[0] / 3) - 1)  # because 0 based
                        between_codons = 1
                    if strand == '+':
                        genome_junction_pos = '{}_{}'.format(pep_modi_coord[1], pep_modi_coord[2])
                    else:
                        genome_junction_pos = '{}_{}'.format(pep_modi_coord[0], pep_modi_coord[3])
                    
                    
                    pep_handle = '>pepID-{};jx_pos-{};between_codons-{};includes_5\'-{};includes_3\'-{};gene-{};jx_coord-{}'.format(
                    pep_idx, aa_junction_pos, between_codons, pep_5include, pep_3include, pep_gene, genome_junction_pos)
                    sp.write(pep_handle + '\n')
                    sp.write(pep_seq + '\n')
               


open /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/TCGA-A2-A0D2-01A-21R-A034-07.all_d4aee54_GTEXcore_kmer_peptides_raw.tsv


In [37]:
pep_orig_coord

['modifiedExonsCoord']

In [49]:
pep_seq

'QSPASPPPLGGGAPVRTAKAERRHQERLRVQSPEPPAPERALSPAELRALEAEKRALWRAARMKSLVLSRSQEGRGTRGPLERLAEAPSPAPTPSPTPVE'

In [60]:
kmer 

'AARMKSLVL'

In [84]:
len(pep_seq) 

100

# GET THE POSITION OF THE JUNCTION IN THE PEPTIDE = NOT The start of the kmer 



In [73]:
kmer

'AQLNILLEV'

In [72]:
pep

['DIIHDPGRGAPLAKVVFRDPYRFKKRTELFIAAEGIHTGQFVYCGKKAQLNILLEVATTSTSASPPPSAEMPLLAAKWVSLLPAGLDVSGEPRLCRRKRTSAEGLNKVCVYAT',
 'ENSG00000161016.17:256_245_8:0:144791914:3-exons_9-mer',
 '1',
 'ENSG00000161016.17',
 'chr8',
 '-',
 'ref',
 '1',
 '0',
 'nan',
 '0',
 'nan',
 'nan',
 '144791772;144791914;144791482;144791495;144789766;144789950',
 '144791772;144791914;144791482;144791495;144789764;144789950',
 '"(256',
 ' 245',
 ' 8)"',
 '3-exons_9-mer\n']

In [60]:
pep_modif_coordS

'144791772;144791914;144791482;144791495;144789764;144789950'

In [61]:
has_stop_codon 

'0'

In [104]:
for pep_modi_coord in pep_modif_coordS.split('/'):
    pep_modi_coord = pep_modi_coord.split(';')
    pep_modi_coord = [coord for coord in pep_modi_coord if coord != 'None']
    tot_len = 0 
    jx_list = []
    shift = 0 
    shift_list = []
    for pair in np.arange(0, len(pep_modi_coord), 2):
        cds = int(pep_modi_coord[pair+ 1]) - int(pep_modi_coord[pair])  # 0 based, open right 
        print(cds)
        cds += shift 
        print(cds)
        shift = cds % 3
        print(shift)
        jx_list.append(cds - shift)
        shift_list.append(shift)
        print(cds - shift)
        print('\n')
    kmer_len = 9 
    for start_pos in np.arange(0, len(pep_seq) - (kmer_len - 1) ):
        if pep_seq[start_pos : start_pos + kmer_len ] == kmer:
            pos_kmer = start_pos # 0 based, real position = pos_kmer +1
    assert(pos_kmer == jx_list[0]/ 3)
    
    if shift_list[0]: # amino acid overlaps the junction 
        nt_junction_pos = jx_list[0]
        between_codons = 0 
    else: # junction between amino acids 
        nt_junction_pos = jx_list[0] - 1
        between_codons = 1
    if strand == '+':
        genome_junction_pos = '{}-{}'.format(pep_modi_coord[0][1], pep_modi_coord[1][0])
    else:
        genome_junction_pos = '{}-{}'.format(pep_modi_coord[0][0], pep_modi_coord[1][1])


142
142
1
141


13
14
2
12


186
188
2
186




In [105]:
shift_list

[1, 2, 2]

In [88]:
sum(jx_list) / 3

113.0

In [89]:
len(pep_seq)

113

In [81]:
142% 3

1

In [82]:
141/3

47.0

In [68]:
(142 + 13 + 186)

341

In [70]:
341 / 3

113.66666666666667

In [71]:
len(pep_seq)

113

In [91]:
113 - 104 

9

In [ ]:
DIIHDPGRGAPLAKVVFRDPYRFKKRTELFIAAEGIHTGQFVYCGKKAQLNILLEVATTSTSASPPPSAEMPLLAAKWVSLLPAGLDVSGEPRLCRRKRTSAEGLNKVCVYAT

In [96]:
pep_seq

'DIIHDPGRGAPLAKVVFRDPYRFKKRTELFIAAEGIHTGQFVYCGKKAQLNILLEVATTSTSASPPPSAEMPLLAAKWVSLLPAGLDVSGEPRLCRRKRTSAEGLNKVCVYAT'

In [97]:
kmer

'AQLNILLEV'

In [99]:
len('DIIHDPGRG')

9

0
DIIHDPGRG
1
IIHDPGRGA
2
IHDPGRGAP
3
HDPGRGAPL
4
DPGRGAPLA
5
PGRGAPLAK
6
GRGAPLAKV
7
RGAPLAKVV
8
GAPLAKVVF
9
APLAKVVFR
10
PLAKVVFRD
11
LAKVVFRDP
12
AKVVFRDPY
13
KVVFRDPYR
14
VVFRDPYRF
15
VFRDPYRFK
16
FRDPYRFKK
17
RDPYRFKKR
18
DPYRFKKRT
19
PYRFKKRTE
20
YRFKKRTEL
21
RFKKRTELF
22
FKKRTELFI
23
KKRTELFIA
24
KRTELFIAA
25
RTELFIAAE
26
TELFIAAEG
27
ELFIAAEGI
28
LFIAAEGIH
29
FIAAEGIHT
30
IAAEGIHTG
31
AAEGIHTGQ
32
AEGIHTGQF
33
EGIHTGQFV
34
GIHTGQFVY
35
IHTGQFVYC
36
HTGQFVYCG
37
TGQFVYCGK
38
GQFVYCGKK
39
QFVYCGKKA
40
FVYCGKKAQ
41
VYCGKKAQL
42
YCGKKAQLN
43
CGKKAQLNI
44
GKKAQLNIL
45
KKAQLNILL
46
KAQLNILLE
47
AQLNILLEV
48
QLNILLEVA
49
LNILLEVAT
50
NILLEVATT
51
ILLEVATTS
52
LLEVATTST
53
LEVATTSTS
54
EVATTSTSA
55
VATTSTSAS
56
ATTSTSASP
57
TTSTSASPP
58
TSTSASPPP
59
STSASPPPS
60
TSASPPPSA
61
SASPPPSAE
62
ASPPPSAEM
63
SPPPSAEMP
64
PPPSAEMPL
65
PPSAEMPLL
66
PSAEMPLLA
67
SAEMPLLAA
68
AEMPLLAAK
69
EMPLLAAKW
70
MPLLAAKWV
71
PLLAAKWVS
72
LLAAKWVSL
73
LAAKWVSLL
74
AAKWVSLLP
75
AKWVSLLPA
76
KWVSLLPAG
77
WVSLLP

In [95]:
pos_kmer * 3

141

In [76]:
pos_kmer * 3 

141

In [39]:
72/3

24.0

In [40]:
119/ 3

39.666666666666664

In [36]:
tot_len / 3

63.666666666666664

In [41]:
len(pep_seq)

46

In [22]:
tot_len / 3 

101.66666666666667

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


In [81]:
pos_kmer

59

In [74]:
len(pep_seq[start_pos : start_pos + kmer_len ] )

9